Copyright Matlantis Corp. as contributors to Matlantis contrib project

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pickle
import h5py
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split

from tqdm import tqdm
from src.dataset import HDF5Dataset

%load_ext autoreload
%autoreload 2

## 3. Prepare 1H dataset
dataset for 1H nmr prediction was prepared.  
The outlier data was removed. (23 data from 1208486 data, which magneticshielding value is under 15)  

In [2]:
#input_file = "dataset/qm9_vapor_pbev8.h5"
input_file = "dataset/qm9_vapor_r2scan.h5"

In [3]:
X = []
y = []
skipped = 0
skipped_other_element = 0

print("Preparing datasets...")
with h5py.File(input_file, "r") as f:
    m_k = list(f.keys())
    for k_id in tqdm(m_k):
        for a_k in f[k_id].keys():
            atoms = f[k_id][a_k]

            if atoms["element"][()] == b"H":
                if atoms["value"][()] < 15:  # remove here and use not filtered data
                    skipped += 1
                    continue
                X.append(atoms["scalar"][()])
                y.append([k_id, atoms["value"][()]])
            else:
                skipped_other_element += 1
                continue

print(f"{skipped} data were skipped in {len(X) + skipped} data")

Preparing datasets...


100%|██████████| 130831/130831 [30:00<00:00, 72.68it/s] 


23 data were skipped in 1208486 data


In [4]:
df_x = pd.DataFrame(X, columns=[f"desc-{x}" for x in range(256)])
df_y = pd.DataFrame(y, columns=["molecule_idx", "value"])

df_all = pd.concat([df_x, df_y], axis=1)
del df_x
del df_y
df_all.to_csv(input_file[:-3]+"_H.csv",index=False)

In [5]:
df_all.shape

(1208463, 258)

In [6]:
df_all.head()

,desc-0,desc-1,desc-2,desc-3,desc-4,desc-5,desc-6,desc-7,desc-8,desc-9,...,desc-248,desc-249,desc-250,desc-251,desc-252,desc-253,desc-254,desc-255,molecule_idx,value
0,0.725259,0.257291,0.238289,-0.157121,0.562056,0.041166,1.364311,0.292885,0.311306,0.320144,...,0.090678,-0.079473,0.432806,1.070554,0.012076,-0.087990,0.165963,-0.098839,m000001,31.5743
1,0.725260,0.257290,0.238290,-0.157120,0.562059,0.041163,1.364309,0.292885,0.311308,0.320145,...,0.090679,-0.079473,0.432805,1.070554,0.012075,-0.087990,0.165964,-0.098838,m000001,31.5743
2,0.725275,0.257281,0.238300,-0.157112,0.562067,0.041146,1.364298,0.292880,0.311308,0.320150,...,0.090683,-0.079477,0.432794,1.070552,0.012067,-0.087986,0.165965,-0.098826,m000001,31.5744
3,0.725273,0.257281,0.238300,-0.157114,0.562064,0.041147,1.364298,0.292879,0.311307,0.320149,...,0.090682,-0.079476,0.432795,1.070552,0.012067,-0.087986,0.165965,-0.098829,m000001,31.5744
4,0.702751,0.412856,0.481192,0.238448,0.789768,0.111927,1.303263,-0.103865,-0.169121,0.390848,...,-0.118581,-0.326023,0.321385,0.600912,0.007953,-0.082237,0.067479,0.129453,m000002,31.9746
